In [73]:
import pandas as pd


df = pd.read_csv("metrics.csv")

# Display the DataFrame
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 24 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   group                   5000 non-null   object 
 1   type                    5000 non-null   object 
 2   pixel_accuracy          5000 non-null   float64
 3   iou                     5000 non-null   float64
 4   dice                    5000 non-null   float64
 5   nsd                     4998 non-null   float64
 6   eroded_pixel_accuracy   5000 non-null   float64
 7   eroded_iou              5000 non-null   float64
 8   eroded_dice             5000 non-null   float64
 9   eroded_nsd              3169 non-null   float64
 10  dilated_pixel_accuracy  5000 non-null   float64
 11  dilated_iou             5000 non-null   float64
 12  dilated_dice            5000 non-null   float64
 13  dilated_nsd             3172 non-null   float64
 14  opened_pixel_accuracy   5000 non-null   

In [7]:
print(df.head(5))

     group    type  pixel_accuracy       iou      dice       nsd  \
0  group_2  manual        0.896872  0.774584  0.872975  0.897516   
1  group_2  manual        0.916192  0.460209  0.630333  0.736391   
2  group_2  manual        0.892232  0.714830  0.833704  0.862381   
3  group_2  manual        0.876864  0.535767  0.697719  0.778443   
4  group_2  manual        0.949328  0.777840  0.875039  0.893461   

   eroded_pixel_accuracy  eroded_iou  eroded_dice  eroded_nsd  ...  \
0               0.866008    0.744922     0.853817    0.001561  ...   
1               0.924684    0.488720     0.656564         NaN  ...   
2               0.901560    0.741187     0.851359    0.000685  ...   
3               0.904441    0.601574     0.751228    0.000000  ...   
4               0.955408    0.811066     0.895678    0.000396  ...   

   opened_pixel_accuracy  opened_iou  opened_dice  opened_nsd  \
0               0.877416    0.765911     0.867440    0.002169   
1               0.917880    0.466634    

In [58]:
# Calculate and display summary statistics
overall_stats = df.groupby('type')[['nsd', 'iou', 'dice', 'time']].agg(['mean', 'std', 'count'])
group_stats = df.groupby(['type', 'group'])[['nsd', 'iou', 'dice', 'time']].agg(['mean', 'std'])

In [70]:
def format_group_stats(stats_df):
    # Create a styled version of the statistics
    styled = stats_df.style\
        .format({
            ('nsd', 'mean'): '{:.2f}', 
            ('nsd', 'std'): '{:.2f}',
            ('iou', 'mean'): '{:.2f}', 
            ('iou', 'std'): '{:.2f}',
            ('dice', 'mean'): '{:.2f}', 
            ('dice', 'std'): '{:.2f}',
            ('time', 'mean'): '{:.0f}', 
            ('time', 'std'): '{:.0f}'
        })\
        .background_gradient(
            subset=pd.IndexSlice[:, [('nsd', 'mean'), ('iou', 'mean'), ('dice', 'mean'), ('time', 'mean')]],
            cmap='RdYlGn'
        )\
        .set_table_styles([
            {'selector': 'th', 'props': [('text-align', 'center')]},
            {'selector': 'td', 'props': [('text-align', 'center')]}
        ])
    
    return styled

# Display the formatted statistics
display(format_group_stats(group_stats))
display(format_group_stats(overall_stats))

As seen in the op_morfologicas.ipynb, the best operation to use on the manual type data is the erosion. 

In [76]:
# Calculate mean and std for 'eroded_nsd', 'eroded_iou', and 'eroded_dice' where type is 'manual'
eroded_stats = df[df['type'] == 'manual'][['eroded_nsd', 'eroded_iou', 'eroded_dice']].agg(['mean', 'std'])

# Update overall_stats with the calculated values
for metric in ['nsd', 'iou', 'dice']:
    overall_stats.loc['manual', (metric, 'mean')] = eroded_stats.loc['mean', f'eroded_{metric}']
    overall_stats.loc['manual', (metric, 'std')] = eroded_stats.loc['std', f'eroded_{metric}']

display(format_group_stats(overall_stats))
